In [1]:
from transformers import set_seed

set_seed(916)

In [1]:
from transformers import BertConfig

BertConfig()

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [2]:
from bert_reduced.bert_reduced import BertReducedForSequenceClassification

# Load the model
model = BertReducedForSequenceClassification.from_pretrained("cayjobla/bert-base-uncased-reduced", revision="pretrain")

Some weights of BertReducedForSequenceClassification were not initialized from the model checkpoint at cayjobla/bert-base-uncased-reduced and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Freze base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

In [3]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("cayjobla/bert-base-uncased-reduced", revision="pretrain")

### COLA

In [4]:
from datasets import load_dataset

# Load the raw data
task = "cola"
raw_datasets = load_dataset("glue", task)

Found cached dataset glue (/home/cayjobla/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Tokenize the raw data
tokenize = lambda batch: tokenizer(batch["sentence"], padding="max_length", truncation=True)
raw_datasets = raw_datasets.map(tokenize, batched=True, desc="Running tokenizer on dataset")

Running tokenizer on dataset:   0%|          | 0/8551 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1043 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [7]:
import evaluate
import numpy as np

def compute_metrics(p, task):
    metric = evaluate.load("glue", task)
    preds = np.argmax(p.predictions, axis=1)
    result = metric.compute(predictions=preds, references=p.label_ids)
    if len(result) > 1:
        result["combined_score"] = np.mean(list(result.values())).item()
    return result

In [8]:
from transformers import TrainingArguments, Trainer, default_data_collator

num_epochs = 5

training_args = TrainingArguments(
    output_dir="bert-base-uncased-reduced",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    push_to_hub=False,
    logging_steps=10,
    run_name="glue-" + task,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=raw_datasets["train"],
    eval_dataset=raw_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

trainer.train()

/home/cayjobla/ReducedBertModel/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cayjobla. Use `wandb login --relogin` to force relogin


OutOfMemoryError: Caught OutOfMemoryError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home/cayjobla/ReducedBertModel/.venv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/cayjobla/ReducedBertModel/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/cayjobla/ReducedBertModel/bert_reduced.py", line 221, in forward
    outputs = self.bert(
  File "/home/cayjobla/ReducedBertModel/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/cayjobla/ReducedBertModel/.venv/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py", line 1020, in forward
    encoder_outputs = self.encoder(
  File "/home/cayjobla/ReducedBertModel/.venv/lib/python3.10/site-packages/torch/nn/modules/modu

: 